In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/brx/brx_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/tam/tam_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/mni/mni_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/urd/urd_test.csv
/kaggle/input/aksharantar/aksharantar_sampled/kok/kok_valid.csv
/kaggle/input/aksharantar/aksharantar_sampled/kok/kok_train.csv
/kaggle/input/aksharantar/aksharantar_sampled/kok/kok_test.csv
/kaggle/input/aksharantar/aksharantar_sampled

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from tqdm import tqdm

import numpy as np
import random
import matplotlib.pyplot as plt

# Instantiates the device to be used as GPU/CPU based on availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [3]:
mal_embedd_size = 24
eng_embedd_size = 31
device.type

'cpu'

In [4]:
arr = np.loadtxt("/kaggle/input/aksharantar/aksharantar_sampled/mal/mal_train.csv",
                 delimiter=",", dtype=str)
num_sample = arr.shape[0]
x_train,y_train = arr[:,0],arr[:,1]
X = np.zeros((num_sample,eng_embedd_size))
y = np.zeros((num_sample,mal_embedd_size))
english_index = 3
mal_index = 3
english_dict = {}
malayalam_dict = {}
english_index_dict = {}
malayalam_index_dict = {}

for i in range(num_sample):
    
    X[i][0] = 1
    y[i][0] = 1
    for j in range(1,len(x_train[i])):
        
        if(english_dict.get(x_train[i][j]) == None):
            english_dict[x_train[i][j]]=english_index
            english_index_dict[english_index] = x_train[i][j]
            english_index+=1
            
            
        X[i][j+1] = english_dict[x_train[i][j]]
    
    X[i][len(x_train[i])]=2
        
    for j in range(len(y_train[i])):
            
        if(malayalam_dict.get(y_train[i][j]) == None):
            malayalam_dict[y_train[i][j]]=mal_index
            malayalam_index_dict[mal_index] = y_train[i][j]
            mal_index+=1
        y[i][j+1] = malayalam_dict[y_train[i][j]]
        
    y[i][len(y_train[i])] = 2

In [5]:
class MakeDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.int64)
        self.y = torch.tensor(y,dtype=torch.int64)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

  
    def __len__(self):
        return self.len

In [6]:
dataset = MakeDataset(X,y)
train_loader = DataLoader(dataset,shuffle=True,batch_size=256)

In [7]:
class Encoder(nn.Module):
    '''
    Simple RNN based encoder network
    '''
    def __init__(self, input_dim, embed_dim, hidden_dim ,
                    rnn_type = 'lstm',
                    layers = 2,
                    bidirectional = False,
                    dropout = 0,
                    device = device
                ):
        
        super(Encoder, self).__init__()

        self.detail_parameters = {
            'input_dim' : input_dim,
            'embed_dim' : embed_dim,
            'hidden_dim' : hidden_dim,
            'rnn_type' : rnn_type,
            'layers' : layers,
            'directions' : 2 if bidirectional else 1,
            'device' : device.type,
        }
        
        self.input_dim = input_dim #src_vocab_sz
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.rnn_type = rnn_type
        self.layers = layers
        self.directions = 2 if bidirectional else 1
        self.device = device


        self.embedding = nn.Embedding(self.input_dim, self.embed_dim)

        if self.rnn_type == 'rnn':
            self.enc_rnn = nn.RNN(input_size= self.embed_dim,
                          hidden_size= self.hidden_dim,
                          num_layers= self.layers,
                          bidirectional= bidirectional)
            
        elif self.rnn_type == "gru":
            self.enc_rnn = nn.GRU(input_size= self.enc_embed_dim,
                          hidden_size= self.enc_hidden_dim,
                          num_layers= self.enc_layers,
                          bidirectional= bidirectional)
        elif self.rnn_type == "lstm":
            self.enc_rnn = nn.LSTM(input_size= self.enc_embed_dim,
                          hidden_size= self.enc_hidden_dim,
                          num_layers= self.enc_layers,
                          bidirectional= bidirectional)
        else:
            raise Exception("unknown RNN type mentioned")
            
            
    def forward(self, input, hidden):
        # Convert input sequence to embeddings

        embedded = self.embedding(input)

        output, hidden = self.enc_rnn(embedded, hidden)

        return output, hidden

    def getParams(self):
        return self.detail_parameters
    
    def init_hidden(self, batch):
        # Initialize the hidden state to zeros
        return torch.zeros(self.layers,batch,self.hidden_dim,device=device)
        
    

In [8]:
# a,b = next(iter(train_loader))
# enc = Encoder(27,64,64)
# o,h=enc.forward(a[:10,:].T,enc.init_hidden(10))

In [9]:
class Decoder(nn.Module):
    '''
    Used as decoder stage
    '''
    def __init__(self, input_dim, embed_dim, hidden_dim,
                    rnn_type = 'lstm',
                    layers = 2,
                    use_attention = False,
                    outstate_dim = None,
                    dropout = 0,
                    device = device
                 ):
        super(Decoder, self).__init__()
        
        self.detail_parameters = {
            'input_dim' : input_dim,
            'embed_dim' : embed_dim,
            'hidden_dim' : hidden_dim,
            'rnn_type' : rnn_type,
            'layers' : layers,
            'device' : device.type
        }

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim
        self.rnn_type = rnn_type
        self.layers = layers
        self.use_attention = use_attention
        self.device = device
        if self.use_attention:
            self.outstate_dim = outstate_dim if outstate_dim else hidden_dim
        else:
            self.outstate_dim = 0


        self.embedding = nn.Embedding(self.input_dim, self.embed_dim)
    
        self.softmax = F.softmax
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_dim, self.embed_dim), nn.LeakyReLU(),
            nn.Linear(self.embed_dim, self.input_dim),
            )

        if self.rnn_type == 'rnn':
            self.dec_rnn = nn.RNN(input_size= self.embed_dim,
                                  
                          hidden_size= self.hidden_dim, # previous Hidden
                          num_layers= self.layers,)
        elif self.rnn_type == 'gru':
            self.dec_rnn = nn.GRU(input_size= self.embed_dim + self.outstate_dim, # to concat attention_output
                          hidden_size= self.hidden_dim, # previous Hidden
                          num_layers= self.layers,
                          batch_first = True )
        elif self.rnn_type == "lstm":
            self.dec_rnn = nn.LSTM(input_size= self.embed_dim + self.outstate_dim, # to concat attention_output
                          hidden_size= self.hidden_dim, # previous Hidden
                          num_layers= self.layers,
                          batch_first = True )
        else:
            raise Exception("unknown RNN type mentioned")


    def getParams(self):
        return self.detail_parameters
    
    def forward(self, input, hidden):
#         print(hidden.shape)
        output = self.embedding(input)

        output, hidden = self.dec_rnn(output, hidden)

        output = self.fc(output)
        
        return output, hidden

    

In [10]:
# ob = Decoder(70,64,64)
# s,d = ob.forward(b[:10,:].T,h)
# s.shape

In [11]:
class Seq2Seq(nn.Module):
    
    def __init__(self, 
                 input_seq_length = 31,
                 output_seq_length = 24,
                 enc_input_dim = 29, 
                 dec_input_dim = 72,
                 enc_hidden_dim = 64, 
                 dec_hidden_dim =64,
                 enc_embed_dim = 64, 
                 dec_embed_dim = 64, 
                 bidirectional = False,
                 layers = 2,
                 rnn_type = 'rnn', 
                 dropout = 0, 
                 device = device
                ):
        
        
        super(Seq2Seq, self).__init__()
        self.detail_parameters = {
         'input_seq_length': input_seq_length,
         'output_seq_length' : output_seq_length,
         'enc_input_dim' : enc_input_dim, 
         'dec_input_dim' : dec_input_dim,
         'enc_embed_dim' : enc_embed_dim, 
         'dec_embed_dim' : dec_embed_dim, 
         'bidirectional' : bidirectional,
         'layers' : layers,
         'rnn_type' :rnn_type, 
         'dropout' : dropout, 
         'device' : device.type
        }
        self.input_seq_length = input_seq_length
        self.output_seq_length = output_seq_length
        self.enc_input_dim = enc_input_dim
        self.dec_input_dim = dec_input_dim
        self.enc_hidden_dim = enc_hidden_dim
        self.dec_hidden_dim = dec_hidden_dim
        self.enc_embed_dim = enc_embed_dim
        self.dec_embed_dim = dec_embed_dim 
        self.direction = bidirectional
        self.layers = layers
        self.rnn_type = rnn_type 
        self.dropout = dropout
        self.device = device
        self.softmax = F.softmax
        
        self.encoder = Encoder(input_dim = self.enc_input_dim,
                               embed_dim = self.enc_embed_dim, 
                               hidden_dim =  self.enc_hidden_dim,
                               rnn_type = self.rnn_type,
                               layers = self.layers,
                               bidirectional = self.direction,
                               dropout = self.dropout, 
                               device = self.device
                              )
        self.decoder = Decoder(
                               input_dim = self.dec_input_dim,
                               embed_dim = self.dec_embed_dim,
                               hidden_dim = self.enc_hidden_dim,
                               rnn_type = self.rnn_type,
                               layers = self.layers,
                               dropout = self.dropout, 
                               device = self.device
                               )
        
    def getParams(self):
        return self.detail_parameters
    
    def forward(self, input):
        
        batch_size = input.shape[0]
        
        enc_hidden = self.encoder.init_hidden(batch_size)
        
        enc_output,enc_last_state = self.encoder.forward(input.T, enc_hidden)

        
        predicted = torch.zeros(self.output_seq_length, batch_size, self.dec_input_dim,device = self.device)
        
        
        dec_hidden = enc_last_state
        
        output = torch.ones(1,batch_size,dtype=torch.long, device=self.device)
        
        for t in range(self.output_seq_length-1):
            output,dec_hidden=self.decoder.forward(output,dec_hidden)
            predicted[t] = output
#             print(output.shape)
            output = self.softmax(output,dim=2)
            output = torch.argmax(output,dim=2)
#             print(output.shape)
            

        return predicted
            
        
        
    

In [12]:
model = Seq2Seq()
model.to(device)
epochs = 20
def train(data_loader, epochs):
        
        optimizer = optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss()

        # set model to train mode
        model.train()
        
        for epoch in tqdm(range(epochs)):
            total_loss=0
            for i, (source, target) in enumerate(data_loader):

                source = source.to(device)
                target = target.to(device)
                
                optimizer.zero_grad()
                
                output = model.forward(source)  # exclude last token from target sequence
                output = output.permute(1, 0, 2)
                
                output = output.reshape(-1, 72)
                target = target.reshape(-1)

                loss = criterion(output, target)  # calculate loss
                
                loss.backward()  # compute gradients
                optimizer.step()  # update parameters
                
                total_loss += loss.item()
                
                
            print(f'epoch {epoch} => loss = {total_loss}')

train(train_loader, epochs)

  5%|▌         | 1/20 [00:20<06:21, 20.08s/it]

epoch 0 => loss = 498.88017559051514


 10%|█         | 2/20 [00:45<06:53, 22.98s/it]

epoch 1 => loss = 430.21292519569397


 15%|█▌        | 3/20 [01:08<06:35, 23.24s/it]

...

epoch 17 => loss = 427.9525020122528


 95%|█████████▌| 19/20 [12:18<00:53, 53.61s/it]

epoch 18 => loss = 427.7858328819275


100%|██████████| 20/20 [13:15<00:00, 39.78s/it]

epoch 19 => loss = 427.7225983142853


In [13]:
def printOutput(y):
    s=''
    for i in range(len(y)):
        if y[i] == 0 or y[i] == 1 or y[i] == 2:
            continue
        s+=malayalam_index_dict[y[i]]
    return s

def accuracy(y_dash , y):
    num_sample,index = y_dash.shape
    score = 0
    for i in range(num_sample):
        flag = 0
        for j in range(index):
            if y_dash[i][j] !=  y[i][j]:
                flag = 1
                break
        if flag == 0:
            score+=1
    return score/num_sample


output = model.forward(train_loader.dataset.x)

output = output.permute(1, 0, 2)

output = F.softmax(output,dim=2)
output = torch.argmax(output,dim=2)
output.shape
# train_loader.dataset.x[0].unsqueeze(1).shape

torch.Size([51200, 24])

In [14]:
# printOutput(train_loader.dataset.y[0])
accuracy(output.T,train_loader.dataset.y)

0.0

In [15]:
print(output[0])
train_loader.dataset.y[0]
#     print(name, type(param))
# for (i,j) in param:
#     print (i)

tensor([ 1, 28,  7,  6,  6,  6,  6,  6,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0])


tensor([ 1,  3,  4,  5,  6,  5,  7,  8,  9, 10,  6, 10,  7,  8,  5,  6,  5,  7,
        11,  2,  0,  0,  0,  0])

In [16]:
def printOutput(y):
    s=''
    for i in range(len(y)):
        if y[i] == 0 or y[i] == 1 or y[i] == 2:
            continue
        s+=malayalam_index_dict[y[i]]
    return s

printOutput(train_loader.dataset.y[0].detach().cpu().numpy())
printOutput(output[0].detach().cpu().numpy())
# malayalam_index_dict[train_loader.dataset.y[0][1]]


'പി്്്്്്'

In [17]:
# a,b = next(iter(train_loader))
# a.shape

# train_loader.dataset.x[0].unsqueeze(0).shape